## Notebook overview
ResNet-1D windowed classifier on engineered features (single/pair), using focal BCE, StratifiedGroupKFold, and per-class threshold tuning.

# Import and configs


In [1]:
!pip install --no-deps /kaggle/input/dependencies/koolbox-0.1.3-py3-none-any.whl


Processing /kaggle/input/dependencies/koolbox-0.1.3-py3-none-any.whl


In [2]:
import json

from collections import defaultdict

import pandas as pd
import polars as pl


class HostVisibleError(Exception):
    pass

def single_lab_f1(lab_solution: pl.DataFrame, lab_submission: pl.DataFrame, beta: float = 1) -> float:
    label_frames: defaultdict[str, set[int]] = defaultdict(set)
    prediction_frames: defaultdict[str, set[int]] = defaultdict(set)

    for row in lab_solution.to_dicts():
        label_frames[row['label_key']].update(range(row['start_frame'], row['stop_frame']))

    for video in lab_solution['video_id'].unique():
        active_labels: str = lab_solution.filter(pl.col('video_id') == video)['behaviors_labeled'].first()  # ty: ignore
        active_labels: set[str] = set(json.loads(active_labels))
        predicted_mouse_pairs: defaultdict[str, set[int]] = defaultdict(set)

        for row in lab_submission.filter(pl.col('video_id') == video).to_dicts():
            # Since the labels are sparse, we can't evaluate prediction keys not in the active labels.
            if ','.join([str(row['agent_id']), str(row['target_id']), row['action']]) not in active_labels:
                continue

            new_frames = set(range(row['start_frame'], row['stop_frame']))
            
            # Ignore truly redundant predictions.
            new_frames = new_frames.difference(prediction_frames[row['prediction_key']])
            prediction_pair = ','.join([str(row['agent_id']), str(row['target_id'])])
            if predicted_mouse_pairs[prediction_pair].intersection(new_frames):
                # A single agent can have multiple targets per frame (ex: evading all other mice) but only one action per target per frame.
                raise HostVisibleError('Multiple predictions for the same frame from one agent/target pair')
            
            prediction_frames[row['prediction_key']].update(new_frames)
            predicted_mouse_pairs[prediction_pair].update(new_frames)

    tps = defaultdict(int)
    fns = defaultdict(int)
    fps = defaultdict(int)
    for key, pred_frames in prediction_frames.items():
        action = key.split('_')[-1]
        matched_label_frames = label_frames[key]
        tps[action] += len(pred_frames.intersection(matched_label_frames))
        fns[action] += len(matched_label_frames.difference(pred_frames))
        fps[action] += len(pred_frames.difference(matched_label_frames))

    distinct_actions = set()
    for key, frames in label_frames.items():
        action = key.split('_')[-1]
        distinct_actions.add(action)
        if key not in prediction_frames:
            fns[action] += len(frames)

    action_f1s = []
    for action in distinct_actions:
        if tps[action] + fns[action] + fps[action] == 0:
            action_f1s.append(0)
        else:
            action_f1s.append((1 + beta**2) * tps[action] / ((1 + beta**2) * tps[action] + beta**2 * fns[action] + fps[action]))
    return sum(action_f1s) / len(action_f1s)

def mouse_fbeta(solution: pd.DataFrame, submission: pd.DataFrame, beta: float = 1) -> float:

    if len(solution) == 0 or len(submission) == 0:
        raise ValueError('Missing solution or submission data')

    expected_cols = ['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']

    for col in expected_cols:
        if col not in solution.columns:
            raise ValueError(f'Solution is missing column {col}')
        if col not in submission.columns:
            raise ValueError(f'Submission is missing column {col}')

    solution: pl.DataFrame = pl.DataFrame(solution)
    submission: pl.DataFrame = pl.DataFrame(submission)
    assert (solution['start_frame'] <= solution['stop_frame']).all()
    assert (submission['start_frame'] <= submission['stop_frame']).all()
    solution_videos = set(solution['video_id'].unique())
    
    # Need to align based on video IDs as we can't rely on the row IDs for handling public/private splits.
    submission = submission.filter(pl.col('video_id').is_in(solution_videos))

    solution = solution.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('label_key'),
    )
    submission = submission.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('prediction_key'),
    )

    lab_scores = []
    for lab in solution['lab_id'].unique():
        lab_solution = solution.filter(pl.col('lab_id') == lab).clone()
        lab_videos = set(lab_solution['video_id'].unique())
        lab_submission = submission.filter(pl.col('video_id').is_in(lab_videos)).clone()
        lab_scores.append(single_lab_f1(lab_solution, lab_submission, beta=beta))

    return sum(lab_scores) / len(lab_scores)

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, beta: float = 1) -> float:
    solution = solution.drop(row_id_column_name, axis='columns', errors='ignore')
    submission = submission.drop(row_id_column_name, axis='columns', errors='ignore')
    return mouse_fbeta(solution, submission, beta=beta)

In [3]:
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import f1_score
from sklearn.base import clone
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from tqdm.notebook import tqdm
from koolbox import Trainer
import numpy as np
import itertools
import optuna
import warnings
import joblib
import glob
import gc
import logging
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)
SEED = 1245

In [4]:

class CFG:
    train_path = "/kaggle/input/MABe-mouse-behavior-detection/train.csv"
    test_path = "/kaggle/input/MABe-mouse-behavior-detection/test.csv"
    train_annotation_path = "/kaggle/input/MABe-mouse-behavior-detection/train_annotation"
    train_tracking_path = "/kaggle/input/MABe-mouse-behavior-detection/train_tracking"
    test_tracking_path = "/kaggle/input/MABe-mouse-behavior-detection/test_tracking"

    model_root = "/kaggle/input"
    model_name = "1d-cnn"
    model_dir = f"{model_root}/{model_name}"

    patience = 5

    mode = "submit"
    # mode = "validate"
    
    n_splits = 3
    cv = StratifiedGroupKFold(n_splits)


# Loading and preprocessing

In [5]:
train = pd.read_csv(CFG.train_path)
test = pd.read_csv(CFG.test_path)

train["n_mice"] = 4 - train[["mouse1_strain", "mouse2_strain", "mouse3_strain", "mouse4_strain"]].isna().sum(axis=1)
train_without_mbae = train.query("~lab_id.str.startswith('MABe22_')")


In [6]:
# get unique raw entries
body_parts_tracked_list = list(np.unique(train.body_parts_tracked))

## Creating label dataframe

In [7]:
def create_solution_df(dataset):
    solution = []
    missing_file = []
    for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
        lab_id = row['lab_id']
        
        if lab_id.startswith('MABe22'): 
            continue
            
        video_id = row['video_id']
        path = f"{CFG.train_annotation_path}/{lab_id}/{video_id}.parquet"
        try:
            anno = pd.read_parquet(path)
        except FileNotFoundError:
            missing_file.append(path)
            continue

        anno['lab_id'] = lab_id
        anno['video_id'] = video_id
        anno['behaviors_labeled'] = row['behaviors_labeled']
        
        anno['target_id'] = np.where(anno.target_id != anno.agent_id, anno['target_id'].apply(lambda s: f"mouse{s}"), 'self')
        anno['agent_id'] = anno['agent_id'].apply(lambda s: f"mouse{s}")

        solution.append(anno)
    
    solution = pd.concat(solution)
    return solution, missing_file

# use method above to create ground truth df
if CFG.mode == 'validate':
    solution, missing = create_solution_df(train_without_mbae)
    logging.warning("Files not found:")
    print(missing)

In [8]:
DROP_BODY_PARTS = [
    'headpiece_bottombackleft', 'headpiece_bottombackright',
    'headpiece_bottomfrontleft', 'headpiece_bottomfrontright',
    'headpiece_topbackleft', 'headpiece_topbackright',
    'headpiece_topfrontleft', 'headpiece_topfrontright',
    'spine_1', 'spine_2',
    'tail_middle_1', 'tail_middle_2', 'tail_midpoint'
]

def generate_mouse_data(dataset, mode=None, is_train=True):
    if is_train:
        data_dir = CFG.train_tracking_path
    else:
        data_dir = CFG.test_tracking_path

    for _, row in dataset.iterrows():
        lab_id = row.lab_id

        # skip if MABe lab or not string behaviors_labeled   
        if lab_id.startswith("MABe22") or not isinstance(row.behaviors_labeled, str):
            continue

        video_id = row.video_id
        tracking_path =  f"{data_dir}/{lab_id}/{video_id}.parquet";

        vid = pd.read_parquet(tracking_path)

        # > 5 bodyparts -> drop
        if len(np.unique(vid.bodypart)) > 5:
            vid = vid.query("~bodypart.isin(@DROP_BODY_PARTS)")

        # pivot
        pvid = vid.pivot(
            index="video_frame",
            columns=["mouse_id", "bodypart"],
            values=["x", "y"],
        )

        # delete vid for memory
        del vid
        gc.collect()

        # (coor, mouse, bodypart) -> (mouse, bodypart, coor) -> sorted columns 
        pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T
        
        # pix to cm
        pvid = pvid / row.pix_per_cm_approx
        
        # behaviors_labeled is JSON list
        raw_behaviors = json.loads(row.behaviors_labeled)
        
        # remove ', duplicate by set then sort
        cleaned = {b.replace("'", "") for b in raw_behaviors} 
        cleaned = sorted(list(cleaned))

        # split into 3 cols
        behaviors_split = [b.split(",") for b in cleaned]
        vid_beh = pd.DataFrame(behaviors_split, columns=["agent", "target", "action"])

        if is_train:
            try: 
                anno_path = tracking_path.replace("train_tracking", "train_annotation")
                anno = pd.read_parquet(anno_path)
            except FileNotFoundError:
                continue
        
        # ---- SINGLE MOUSE ----
        if mode is None or mode == 'single' :
            # only get target == self
            vid_beh_single = vid_beh.query("target == 'self'")

            for agent_str in np.unique(vid_beh_single.agent):
                try:
                    # get the id (the last element of agent_str)
                    mouse_id = int(agent_str[-1])
                    
                    # get all action of this agent 
                    agent_actions = np.unique(vid_beh_single.query("agent == @agent_str").action)

                    # get tracking of this agent
                    single_mouse = pvid.loc[:, mouse_id] 
                    assert len(single_mouse) == len(pvid)
                
                    single_meta = pd.DataFrame({
                        "video_id": video_id,
                        "agent_id": agent_str,
                        "target_id": "self",
                        "video_frame": single_mouse.index, # index by frames
                    })

                    if is_train:
                        single_label = pd.DataFrame(0.0, columns=agent_actions, index=single_mouse.index)
                        anno_single = anno.query("(agent_id == @mouse_id) & (target_id == @mouse_id)")

                        for _, anno_row in anno_single.iterrows():
                            start = anno_row['start_frame']
                            end = anno_row['stop_frame']
                            action = anno_row['action']
                            single_label.loc[start:end, action] = 1.0

                        yield "single", single_mouse, single_meta, single_label

                    else:
                        yield "single", single_mouse, single_meta, agent_actions
                    
                except KeyError:
                    continue
        
        # ---- PAIR MOUSE ----
        if mode is None or mode == 'pair':
            # only get target != 'self'
            vid_behaviors_pair = vid_beh.query("target != 'self'")

            if len(vid_behaviors_pair) == 0:
                continue

            # get list of mouse_ids
            mouse_ids = np.unique(pvid.columns.get_level_values("mouse_id"))

            # permutation (agent, target) with agent != target
            for agent_id, target_id in itertools.permutations(mouse_ids, 2):
                agent_str = f"mouse{agent_id}"
                target_str = f"mouse{target_id}"

                # action of this (agent, target)
                pair_actions = np.unique(
                    vid_behaviors_pair.query("(agent == @agent_str) & (target == @target_str)").action
                )

                # tracking of these 2 mice
                mouse_pair = pd.concat(
                    [pvid[agent_id], pvid[target_id]],
                    axis=1,
                    keys=["A", "B"],  # A = agent, B = target
                )
                assert len(mouse_pair) == len(pvid)

                # metadata 
                pair_meta = pd.DataFrame({
                    "video_id": video_id,
                    "agent_id": agent_str,
                    "target_id": target_str,
                    "video_frame": mouse_pair.index,
                })

                if is_train:
                    # label: frame × action
                    pair_label = pd.DataFrame(0.0, columns=pair_actions, index=mouse_pair.index)
                    anno_pair = anno.query(
                        "(agent_id == @agent_id) & (target_id == @target_id)"
                    )

                    for _, anno_row in anno_pair.iterrows():
                        start = anno_row["start_frame"]
                        end = anno_row["stop_frame"]
                        action = anno_row["action"]
                        pair_label.loc[start:end, action] = 1.0

                    yield "pair", mouse_pair, pair_meta, pair_label

                else:
                    # test/val: list action
                    yield "pair", mouse_pair, pair_meta, pair_actions
    

# Feature Engineering


In [9]:
# helper

def scaled_window(n_frames_30fps , fps , min_frac=0.2, min_abs=1):
    ws = max(1, int(round(n_frames_30fps * float(fps) / 30.0)))
    min_periods = max(min_abs, int(round(ws * min_frac)))
    return ws, min_periods


def _fps_from_meta(meta_df, fallback_lookup: dict, default_fps: float = 30.0):
    if "frames_per_second" in meta_df.columns and pd.notnull(meta_df["frames_per_second"]).any():
        return float(meta_df["frames_per_second"].iloc[0])
    vid = meta_df["video_id"].iloc[0]
    return float(fallback_lookup.get(vid, default_fps))

def _scale(n_frames_at_30fps, fps, ref=30.0):
    return max(1, int(round(n_frames_at_30fps * float(fps) / ref)))

def _scale_signed(n_frames_at_30fps, fps, ref=30.0):
    if n_frames_at_30fps == 0:
        return 0
    s = 1 if n_frames_at_30fps > 0 else -1
    mag = max(1, int(round(abs(n_frames_at_30fps) * float(fps) / ref)))
    return s * mag

In [10]:
# feature for each mouse

def add_curvature_features(X, center_x, center_y, fps):
    # velocity & acceleration
    vx = center_x.diff()
    vy = center_y.diff()
    ax = vx.diff()
    ay = vy.diff()

    # curve ~ |v × a| / |v|^3
    cross = vx * ay - vy * ax
    speed = np.sqrt(vx**2 + vy**2)
    curvature = np.abs(cross) / (speed**3 + 1e-6)

    # avg curve in scales
    for base_w in [25, 50, 75]:
        ws, mp = scaled_window(base_w, fps, min_frac=0.2)
        X[f"curv_mean_{base_w}"] = curvature.rolling(ws, min_periods=mp).mean()
    
    angle = np.arctan2(vy, vx)
    angle_change = np.abs(angle.diff())
    ws, mp = scaled_window(30, fps, min_frac=0.2)
    X["turn_rate_30"] = angle_change.rolling(ws, min_periods=mp).sum()

    return X

def add_multiscale_features(X, center_x, center_y, fps):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)

    scales = [20, 40, 60, 80]
    for base_w in scales:
        ws, mp = scaled_window(base_w, fps, min_frac=0.25)
        if len(speed) >= ws:
            X[f"sp_m{base_w}"] = speed.rolling(ws, min_periods=mp).mean()
            X[f"sp_s{base_w}"] = speed.rolling(ws, min_periods=mp).std()

    if all(f"sp_m{s}" in X.columns for s in (scales[0], scales[-1])):
        X["sp_ratio"] = X[f"sp_m{scales[0]}"] / (X[f"sp_m{scales[-1]}"] + 1e-6)

    return X

def add_state_features(X, center_x, center_y, fps):
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)

    ws_ma, mp_ma = scaled_window(15, fps, min_frac=1/3)
    speed_ma = speed.rolling(ws_ma, min_periods=mp_ma).mean()

    try:
        bins = [-np.inf, 0.5 * fps, 2.0 * fps, 5.0 * fps, np.inf]
        speed_states = pd.cut(speed_ma, bins=bins, labels=[0, 1, 2, 3]).astype(float)

        for base_w in [20, 40, 60, 80]:
            ws, mp = scaled_window(base_w, fps, min_frac=0.2)
            if len(speed_states) < ws:
                continue

            for state in [0, 1, 2, 3]:
                X[f"s{state}_{base_w}"] = (
                    (speed_states == state)
                    .astype(float)
                    .rolling(ws, min_periods=mp)
                    .mean()
                )

            state_changes = (speed_states != speed_states.shift(1)).astype(float)
            X[f"trans_{base_w}"] = state_changes.rolling(ws, min_periods=mp).sum()
    except Exception:
        pass

    return X

def add_longrange_features(X, center_x, center_y, fps):
    # longrange moving average
    for base_w in [30, 60, 120]:
        ws, mp = scaled_window(base_w, fps, min_frac=1/6, min_abs=5)
        if len(center_x) >= ws:
            X[f"x_ml{base_w}"] = center_x.rolling(ws, min_periods=mp).mean()
            X[f"y_ml{base_w}"] = center_y.rolling(ws, min_periods=mp).mean()
        
    # EWMA 
    for span in [30, 60, 120]:
        s, _ = scaled_window(span, fps, min_frac=0.0)  # min_periods sẽ set riêng
        X[f"x_e{span}"] = center_x.ewm(span=s, min_periods=1).mean()
        X[f"y_e{span}"] = center_y.ewm(span=s, min_periods=1).mean()

    # percentile rank of speed
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    for base_w in [30, 60, 120]:
        ws, mp = scaled_window(base_w, fps, min_frac=1 / 6, min_abs=5)
        if len(speed) >= ws:
            X[f"sp_pct{base_w}"] = speed.rolling(ws, min_periods=mp).rank(pct=True)

    return X


def add_cumulative_distance_single(X, cx, cy, fps, horizon_frames_base: int = 180, colname: str = "path_cum180"):
    L = max(1, _scale(horizon_frames_base, fps))  # frames
    # step length (cm per frame since coords are cm)
    step = np.hypot(cx.diff(), cy.diff())
    # centered rolling sum over ~2L+1 frames (acausal)
    path = step.rolling(2*L + 1, min_periods=max(5, L//6), center=True).sum()
    X[colname] = path.fillna(0.0).astype(np.float32)
    return X


def add_groom_microfeatures(X, df, fps):
    parts = df.columns.get_level_values(0)
    if 'body_center' not in parts or 'nose' not in parts:
        return X

    cx = df['body_center']['x']; cy = df['body_center']['y']
    nx = df['nose']['x']; ny = df['nose']['y']

    cs = (np.sqrt(cx.diff()**2 + cy.diff()**2) * float(fps)).fillna(0)
    ns = (np.sqrt(nx.diff()**2 + ny.diff()**2) * float(fps)).fillna(0)

    w30 = _scale(30, fps)
    X['head_body_decouple'] = (ns / (cs + 1e-3)).clip(0, 10).rolling(w30, min_periods=max(1, w30//3)).median()

    r = np.sqrt((nx - cx)**2 + (ny - cy)**2)
    X['nose_rad_std'] = r.rolling(w30, min_periods=max(1, w30//3)).std().fillna(0)

    if 'tail_base' in parts:
        ang = np.arctan2(df['nose']['y']-df['tail_base']['y'], df['nose']['x']-df['tail_base']['x'])
        dang = np.abs(ang.diff()).fillna(0)
        X['head_orient_jitter'] = dang.rolling(w30, min_periods=max(1, w30//3)).mean()

    return X

# ===============================================================
# 1) Past–vs–Future speed asymmetry (acausal, continuous)
#    Δv = mean_future(speed) - mean_past(speed)
# ===============================================================
def add_speed_asymmetry_future_past_single(
    X: pd.DataFrame, cx: pd.Series, cy: pd.Series, fps: float,
    horizon_base: int = 30, agg: str = "mean"
) -> pd.DataFrame:
    w = max(3, _scale(horizon_base, fps))
    v = _speed(cx, cy, fps)
    if agg == "median":
        v_past = v.rolling(w, min_periods=max(3, w//4), center=False).median()
        v_fut  = v.iloc[::-1].rolling(w, min_periods=max(3, w//4)).median().iloc[::-1]
    else:
        v_past = v.rolling(w, min_periods=max(3, w//4), center=False).mean()
        v_fut  = _roll_future_mean(v, w, min_p=max(3, w//4))
    X["spd_asym_1s"] = (v_fut - v_past).fillna(0.0)
    return X

# ===============================================================
# 2) Distribution shift (future vs past) via symmetric KL of
#    Gaussian fits on speed 
# ===============================================================

def _speed(cx: pd.Series, cy: pd.Series, fps: float) -> pd.Series:
    return np.hypot(cx.diff(), cy.diff()).fillna(0.0) * float(fps)

def _roll_future_mean(s: pd.Series, w: int, min_p: int = 1) -> pd.Series:
    # mean over [t, t+w-1]
    return s.iloc[::-1].rolling(w, min_periods=min_p).mean().iloc[::-1]

def _roll_future_var(s: pd.Series, w: int, min_p: int = 2) -> pd.Series:
    # var over [t, t+w-1]
    return s.iloc[::-1].rolling(w, min_periods=min_p).var().iloc[::-1]


def add_gauss_shift_speed_future_past_single(
    X: pd.DataFrame, cx: pd.Series, cy: pd.Series, fps: float,
    window_base: int = 30, eps: float = 1e-6
) -> pd.DataFrame:
    w = max(5, _scale(window_base, fps))
    v = _speed(cx, cy, fps)

    mu_p = v.rolling(w, min_periods=max(3, w//4)).mean()
    va_p = v.rolling(w, min_periods=max(3, w//4)).var().clip(lower=eps)

    mu_f = _roll_future_mean(v, w, min_p=max(3, w//4))
    va_f = _roll_future_var(v, w, min_p=max(3, w//4)).clip(lower=eps)

    # KL(Np||Nf) + KL(Nf||Np)
    kl_pf = 0.5 * ((va_p/va_f) + ((mu_f - mu_p)**2)/va_f - 1.0 + np.log(va_f/va_p))
    kl_fp = 0.5 * ((va_f/va_p) + ((mu_p - mu_f)**2)/va_p - 1.0 + np.log(va_p/va_f))
    X["spd_symkl_1s"] = (kl_pf + kl_fp).replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return X


In [11]:
def add_single_extra_features(X, single_mouse, available_parts, fps):
    # posture curvature
    if all(p in available_parts for p in ['nose', 'body_center', 'tail_base']):
        # body_center - tail_base vs nose - body_center
        v1 = single_mouse['body_center'] - single_mouse['tail_base']
        v2 = single_mouse['nose'] - single_mouse['body_center']

        dot = v1['x'] * v2['x'] + v1['y'] * v2['y']
        n1 = np.sqrt(v1['x']**2 + v1['y']**2)
        n2 = np.sqrt(v2['x']**2 + v2['y']**2)

        X['pose_curv'] = (dot / (n1 * n2 + 1e-6)).astype(np.float32)

    # verticality/ rearing proxy
    if all(p in available_parts for p in ['nose', 'lateral_left', 'lateral_right']):
        nose_x = single_mouse['nose']['x']
        nose_y = single_mouse['nose']['y']
        lat_x = (single_mouse['lateral_left']['x'] + single_mouse['lateral_right']['x']) / 2.0
        lat_y = (single_mouse['lateral_left']['y'] + single_mouse['lateral_right']['y']) / 2.0

        nose_lat_dist = np.sqrt((nose_x - lat_x)**2 + (nose_y - lat_y)**2)
        X["nose_lateral_dist"] = nose_lat_dist.astype(np.float32)
        X["nose_lateral_vel"] = nose_lat_dist.diff().astype(np.float32)

    # ear dynamics
    if all(p in available_parts for p in ['ear_left', 'ear_right']):
        ear_d = np.sqrt(
            (single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 +
            (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2
        )
        X["ear_vel"] = ear_d.diff().astype(np.float32)
        X["ear_acc"] = ear_d.diff().diff().astype(np.float32)

    return X

In [12]:
def transform_single(single_mouse, body_parts_tracked, fps):
    available_parts = single_mouse.columns.get_level_values(0)

    # pairwise distance between body part (p1, p2) ---
    features = {}

    for p1, p2 in itertools.combinations(body_parts_tracked, 2):
        if p1 in available_parts and p2 in available_parts:
            diff = single_mouse[p1] - single_mouse[p2]      # (x,y) or (x,y,...) by frame
            dist2 = np.square(diff).sum(axis=1, skipna=False)
            features[f"{p1}+{p2}"] = dist2

    X = pd.DataFrame(features)

    # ensure order
    full_cols = [f"{p1}+{p2}" for p1, p2 in itertools.combinations(body_parts_tracked, 2)]
    X = X.reindex(columns=full_cols, copy=False)

    # raw speed by ear and tail (lag ~10 frame) ---
    if all(p in available_parts for p in ['ear_left', 'ear_right', 'tail_base']):
        lag = _scale(10, fps)
        past = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(lag)

        sp_lf  = np.square(single_mouse['ear_left']  - past['ear_left']).sum(axis=1, skipna=False)
        sp_rt  = np.square(single_mouse['ear_right'] - past['ear_right']).sum(axis=1, skipna=False)
        sp_lf2 = np.square(single_mouse['ear_left']  - past['tail_base']).sum(axis=1, skipna=False)
        sp_rt2 = np.square(single_mouse['ear_right'] - past['tail_base']).sum(axis=1, skipna=False)

        X[['sp_lf', 'sp_rt', 'sp_lf2', 'sp_rt2']] = np.column_stack([sp_lf, sp_rt, sp_lf2, sp_rt2])

    # elongation
    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)

    # body angle
    if all(p in available_parts for p in ['nose', 'body_center', 'tail_base']):
        v1 = single_mouse['nose']      - single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']

        dot = v1['x'] * v2['x'] + v1['y'] * v2['y']
        norm1 = np.sqrt(v1['x']**2 + v1['y']**2)
        norm2 = np.sqrt(v2['x']**2 + v2['y']**2)
        body_ang = dot / (norm1 * norm2 + 1e-6)

        X['body_ang'] = body_ang.astype(np.float32)
        X['body_ang_vel'] = body_ang.diff().astype(np.float32)
        X['body_ang_acc'] = body_ang.diff().diff().astype(np.float32)

    # features by body_center
    if 'body_center' in available_parts:
        cx = single_mouse['body_center']['x']
        cy = single_mouse['body_center']['y']

        for base_w in [5, 15, 30, 60]:
            ws = _scale(base_w, fps)
            roll_kwargs = dict(window=ws, min_periods=1, center=True)

            X[f'cx_m{base_w}'] = cx.rolling(**roll_kwargs).mean()
            X[f'cy_m{base_w}'] = cy.rolling(**roll_kwargs).mean()
            X[f'cx_s{base_w}'] = cx.rolling(**roll_kwargs).std()
            X[f'cy_s{base_w}'] = cy.rolling(**roll_kwargs).std()

            X[f'x_rng{base_w}'] = cx.rolling(**roll_kwargs).max() - cx.rolling(**roll_kwargs).min()
            X[f'y_rng{base_w}'] = cy.rolling(**roll_kwargs).max() - cy.rolling(**roll_kwargs).min()

            # displacement & activity (from diff)
            dx = cx.diff()
            dy = cy.diff()
            disp = np.sqrt(dx.rolling(ws, min_periods=1).sum()**2 + dy.rolling(ws, min_periods=1).sum()**2)
            act = np.sqrt(dx.rolling(ws, min_periods=1).var() + dy.rolling(ws, min_periods=1).var())
            X[f'disp{base_w}'] = disp
            X[f'act{base_w}']  = act

        # advanced feature
        X = add_curvature_features(X, cx, cy, fps)
        X = add_multiscale_features(X, cx, cy, fps)
        X = add_state_features(X, cx, cy, fps)
        X = add_longrange_features(X, cx, cy, fps)
        # X = add_cumulative_distance_single(X, cx, cy, fps, horizon_frames_base=180)
        # X = add_groom_microfeatures(X, single_mouse, fps)
        # X = add_speed_asymmetry_future_past_single(X, cx, cy, fps, horizon_base=30)         
        # X = add_gauss_shift_speed_future_past_single(X, cx, cy, fps, window_base=30)

    # nose–tail_based distance by time 
    if all(p in available_parts for p in ['nose', 'tail_base']):
        nt = np.sqrt(
            (single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
            (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2
        )
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nt_lg{lag}'] = nt.shift(l)
            X[f'nt_df{lag}'] = nt - nt.shift(l)

    # ear distance & consistency
    if all(p in available_parts for p in ['ear_left', 'ear_right']):
        ear_d = np.sqrt(
            (single_mouse['ear_left']['x']   - single_mouse['ear_right']['x'])**2 +
            (single_mouse['ear_left']['y']   - single_mouse['ear_right']['y'])**2
        )
        # offset
        for off in [-30, -20, -10, 10, 20, 30]:
            o = _scale_signed(off, fps)
            X[f'ear_o{off}'] = ear_d.shift(-o)

        w = _scale(30, fps)
        roll_c = dict(window=w, min_periods=1, center=True)
        ear_mean = ear_d.rolling(**roll_c).mean()
        ear_std  = ear_d.rolling(**roll_c).std()
        X['ear_con'] = ear_std / (ear_mean + 1e-6)

    X = add_single_extra_features(X, single_mouse, available_parts, fps)

    return X.astype(np.float32, copy=False)


In [13]:
# feature for mice interaction
def add_interaction_features(X, mouse_pair, avail_A, avail_B, fps):
    if "body_center" not in avail_A or "body_center" not in avail_B:
        return X
    
    # coor
    Ax = mouse_pair["A"]["body_center"]["x"]
    Ay = mouse_pair["A"]["body_center"]["y"]
    Bx = mouse_pair["B"]["body_center"]["x"]
    By = mouse_pair["B"]["body_center"]["y"]

    # relative 
    rel_x = Ax - Bx
    rel_y = Ay - By
    rel_dist = np.sqrt(rel_x**2 + rel_y**2)

    # velocity
    A_vx = Ax.diff()
    A_vy = Ay.diff()
    B_vx = Bx.diff()
    B_vy = By.diff()

    # cosine angle between vector (A, B) and velocity vector
    A_lead = (A_vx * rel_x + A_vy * rel_y) / (np.sqrt(A_vx**2 + A_vy**2) * rel_dist + 1e-6)
    B_lead = (B_vx * (-rel_x) + B_vy * (-rel_y)) / (np.sqrt(B_vx**2 + B_vy**2) * rel_dist + 1e-6)

    # avg of A_lead, B_lead on windows
    for base_w in [30, 60]:
        ws, mp = scaled_window(base_w, fps, min_frac=1 / 6)
        X[f"A_ld{base_w}"] = A_lead.rolling(ws, min_periods=mp).mean()
        X[f"B_ld{base_w}"] = B_lead.rolling(ws, min_periods=mp).mean()
    
    # approach
    approach = -rel_dist.diff()
    chase = approach * B_lead
    ws, mp = scaled_window(30, fps, min_frac=1/6)
    X["chase_30"] = chase.rolling(ws, min_periods=mp).mean()

    # correlation of 2 mice speed in long windows
    A_sp = np.sqrt(A_vx**2 + A_vy**2)
    B_sp = np.sqrt(B_vx**2 + B_vy**2)
    for base_w in [60, 120]:
        ws, mp = scaled_window(base_w, fps, min_frac=1 / 6)
        X[f"sp_cor{base_w}"] = A_sp.rolling(ws, min_periods=mp).corr(B_sp)

    return X


In [14]:
def add_egocentric_interaction_features(X, mouse_pair, avail_A, avail_B, fps):
    # cjeck condition
    ok_A = all(p in avail_A for p in ['nose', 'tail_base', 'body_center'])
    ok_B = all(p in avail_B for p in ['nose', 'tail_base', 'body_center'])
    if not (ok_A and ok_B):
        return X

    # position
    Ax = mouse_pair['A']['body_center']['x']
    Ay = mouse_pair['A']['body_center']['y']
    Bx = mouse_pair['B']['body_center']['x']
    By = mouse_pair['B']['body_center']['y']

    # head direction of A/B
    headA_x = mouse_pair['A']['nose']['x'] - mouse_pair['A']['tail_base']['x']
    headA_y = mouse_pair['A']['nose']['y'] - mouse_pair['A']['tail_base']['y']
    headB_x = mouse_pair['B']['nose']['x'] - mouse_pair['B']['tail_base']['x']
    headB_y = mouse_pair['B']['nose']['y'] - mouse_pair['B']['tail_base']['y']

    # vector A → B
    relAB_x = Bx - Ax
    relAB_y = By - Ay

    # cos / sin angle between A and vector A→B
    dotA = headA_x * relAB_x + headA_y * relAB_y
    norm_headA = np.sqrt(headA_x**2 + headA_y**2) + 1e-6
    norm_relAB = np.sqrt(relAB_x**2 + relAB_y**2) + 1e-6

    cos_bearing_A = dotA / (norm_headA * norm_relAB)
    # sign of cross product → trái/phải
    crossA = headA_x * relAB_y - headA_y * relAB_x
    sin_bearing_A = crossA / (norm_headA * norm_relAB)

    X['A_bearing_cos'] = cos_bearing_A.astype(np.float32)
    X['A_bearing_sin'] = sin_bearing_A.astype(np.float32)

    # for B
    relBA_x = Ax - Bx
    relBA_y = Ay - By
    dotB = headB_x * relBA_x + headB_y * relBA_y
    norm_headB = np.sqrt(headB_x**2 + headB_y**2) + 1e-6
    norm_relBA = np.sqrt(relBA_x**2 + relBA_y**2) + 1e-6

    cos_bearing_B = dotB / (norm_headB * norm_relBA)
    crossB = headB_x * relBA_y - headB_y * relBA_x
    sin_bearing_B = crossB / (norm_headB * norm_relBA)

    X['B_bearing_cos'] = cos_bearing_B.astype(np.float32)
    X['B_bearing_sin'] = sin_bearing_B.astype(np.float32)

    # rolling stats 
    for base_w in [15, 30]:
        ws = _scale(base_w, fps)
        roll = dict(window=ws, min_periods=1, center=True)
        X[f'A_bearing_cos_m{base_w}'] = X['A_bearing_cos'].rolling(**roll).mean()
        X[f'A_bearing_sin_m{base_w}'] = X['A_bearing_sin'].rolling(**roll).mean()
        X[f'B_bearing_cos_m{base_w}'] = X['B_bearing_cos'].rolling(**roll).mean()
        X[f'B_bearing_sin_m{base_w}'] = X['B_bearing_sin'].rolling(**roll).mean()

    return X


In [15]:
def add_asymmetry_features(X, mouse_pair, avail_A, avail_B, fps):
    # need body_center to define speeds
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X

    Ax = mouse_pair['A']['body_center']['x']
    Ay = mouse_pair['A']['body_center']['y']
    Bx = mouse_pair['B']['body_center']['x']
    By = mouse_pair['B']['body_center']['y']

    # velocities (frame-wise differences)
    A_vx = Ax.diff()
    A_vy = Ay.diff()
    B_vx = Bx.diff()
    B_vy = By.diff()

    # instantaneous speeds
    A_sp = np.sqrt(A_vx**2 + A_vy**2)
    B_sp = np.sqrt(B_vx**2 + B_vy**2)

    # asymmetry: difference and ratio
    sp_diff = A_sp - B_sp
    sp_ratio = A_sp / (B_sp + 1e-6)

    X['sp_diff_inst'] = sp_diff.astype(np.float32)
    X['sp_ratio_inst'] = sp_ratio.astype(np.float32)

    # rolling stats over short/medium windows
    for base_w in [30, 60]:
        ws = _scale(base_w, fps)
        roll = dict(window=ws, min_periods=1, center=True)

        X[f'sp_diff_m{base_w}'] = (
            sp_diff.rolling(**roll).mean().astype(np.float32)
        )
        X[f'sp_ratio_m{base_w}'] = (
            sp_ratio.rolling(**roll).mean().astype(np.float32)
        )

    return X


In [16]:
def transform_pair(mouse_pair, body_parts_tracked, fps):
    avail_A = mouse_pair['A'].columns.get_level_values(0)
    avail_B = mouse_pair['B'].columns.get_level_values(0)

    # pairwise distance between A[p1] and B[p2]
    features = {}
    for p1, p2 in itertools.product(body_parts_tracked, repeat=2):
        if p1 in avail_A and p2 in avail_B:
            diff = mouse_pair['A'][p1] - mouse_pair['B'][p2]
            dist2 = np.square(diff).sum(axis=1, skipna=False)
            features[f"12+{p1}+{p2}"] = dist2

    X = pd.DataFrame(features)
    full_cols = [f"12+{p1}+{p2}" for p1, p2 in itertools.product(body_parts_tracked, repeat=2)]
    X = X.reindex(columns=full_cols, copy=False)

    # ear-left speed A/B (lag ~10 frame)
    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        lag = _scale(10, fps)
        shA = mouse_pair['A']['ear_left'].shift(lag)
        shB = mouse_pair['B']['ear_left'].shift(lag)

        sp_A  = np.square(mouse_pair['A']['ear_left'] - shA).sum(axis=1, skipna=False)
        sp_AB = np.square(mouse_pair['A']['ear_left'] - shB).sum(axis=1, skipna=False)
        sp_B  = np.square(mouse_pair['B']['ear_left'] - shB).sum(axis=1, skipna=False)

        X[['sp_A', 'sp_AB', 'sp_B']] = np.column_stack([sp_A, sp_AB, sp_B])

    # elong = dist(nose, tail_base) / dist(ear_left, ear_right)
    have_A_elong = all(p in avail_A for p in ['nose', 'tail_base', 'ear_left', 'ear_right'])
    have_B_elong = all(p in avail_B for p in ['nose', 'tail_base', 'ear_left', 'ear_right'])

    if have_A_elong:
        nose_A = mouse_pair['A']['nose']
        tail_A = mouse_pair['A']['tail_base']
        el_A_l = mouse_pair['A']['ear_left']
        el_A_r = mouse_pair['A']['ear_right']

        nose_tail_A = np.square(nose_A - tail_A).sum(axis=1, skipna=False)
        ear_dist_A  = np.square(el_A_l - el_A_r).sum(axis=1, skipna=False)
        X['elong_A'] = nose_tail_A / (ear_dist_A + 1e-6)

    if have_B_elong:
        nose_B = mouse_pair['B']['nose']
        tail_B = mouse_pair['B']['tail_base']
        el_B_l = mouse_pair['B']['ear_left']
        el_B_r = mouse_pair['B']['ear_right']

        nose_tail_B = np.square(nose_B - tail_B).sum(axis=1, skipna=False)
        ear_dist_B  = np.square(el_B_l - el_B_r).sum(axis=1, skipna=False)
        X['elong_B'] = nose_tail_B / (ear_dist_B + 1e-6)

    # diff and ratio
    if 'elong_A' in X.columns and 'elong_B' in X.columns:
        X['elong_diff']  = X['elong_A'] - X['elong_B']
        X['elong_ratio'] = X['elong_A'] / (X['elong_B'] + 1e-6)

    # relative body angle between A and B
    if all(p in avail_A for p in ['nose', 'tail_base']) and all(p in avail_B for p in ['nose', 'tail_base']):
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']

        dot = dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y']
        nA = np.sqrt(dir_A['x']**2 + dir_A['y']**2)
        nB = np.sqrt(dir_B['x']**2 + dir_B['y']**2)
        X['rel_ori'] = dot / (nA * nB + 1e-6)

    # nose-nose approach
    if 'nose' in avail_A and 'nose' in avail_B:
        nn_cur = np.square(mouse_pair['A']['nose'] - mouse_pair['B']['nose']).sum(axis=1, skipna=False)
        lag = _scale(10, fps)
        shA_n = mouse_pair['A']['nose'].shift(lag)
        shB_n = mouse_pair['B']['nose'].shift(lag)
        nn_past = np.square(shA_n - shB_n).sum(axis=1, skipna=False)
        X['appr'] = nn_cur - nn_past

    # distance categories by body_center
    if 'body_center' in avail_A and 'body_center' in avail_B:
        Ax = mouse_pair['A']['body_center']['x']
        Ay = mouse_pair['A']['body_center']['y']
        Bx = mouse_pair['B']['body_center']['x']
        By = mouse_pair['B']['body_center']['y']

        cd = np.sqrt((Ax - Bx)**2 + (Ay - By)**2)

        X['v_cls'] = (cd < 5.0).astype(float)
        X['cls']   = ((cd >= 5.0)  & (cd < 15.0)).astype(float)
        X['med']   = ((cd >= 15.0) & (cd < 30.0)).astype(float)
        X['far']   = (cd >= 30.0).astype(float)

        # stats on squared distance
        cd2 = np.square(mouse_pair['A']['body_center'] - mouse_pair['B']['body_center']).sum(axis=1, skipna=False)
        
        for base_w in [5, 15, 30, 60]:
            ws = _scale(base_w, fps)
            roll_c = dict(window=ws, min_periods=1, center=True)
    
            X[f'd_m{base_w}']  = cd2.rolling(**roll_c).mean()
            X[f'd_s{base_w}']  = cd2.rolling(**roll_c).std()
            X[f'd_mn{base_w}'] = cd2.rolling(**roll_c).min()
            X[f'd_mx{base_w}'] = cd2.rolling(**roll_c).max()
    
            d_var = cd2.rolling(**roll_c).var()
            X[f'int{base_w}'] = 1.0 / (1.0 + d_var)
    
            # dot product vận tốc body_center
            Axd = Ax.diff()
            Ayd = Ay.diff()
            Bxd = Bx.diff()
            Byd = By.diff()
            coord = Axd * Bxd + Ayd * Byd
            X[f'co_m{base_w}'] = coord.rolling(**roll_c).mean()
            X[f'co_s{base_w}'] = coord.rolling(**roll_c).std()

            # cosine similarity A,B speed (offset) ---
            Avx = Ax.diff()
            Avy = Ay.diff()
            Bvx = Bx.diff()
            Bvy = By.diff()
            vel_cos = (Avx * Bvx + Avy * Bvy) / (
                np.sqrt(Avx**2 + Avy**2) * np.sqrt(Bvx**2 + Bvy**2) + 1e-6
            )
        
            for off in [-30, -20, -10, 0, 10, 20, 30]:
                o = _scale_signed(off, fps)
                X[f'va_{off}'] = vel_cos.shift(-o)
        
            w = _scale(30, fps)
            roll_c30 = dict(window=w, min_periods=1, center=True)
            cd2_mean = cd2.rolling(**roll_c30).mean()
            cd2_std  = cd2.rolling(**roll_c30).std()
            X['int_con'] = cd2_std / (cd2_mean + 1e-6)

            # advanced features
            X = add_asymmetry_features(X, mouse_pair, avail_A, avail_B, fps)
            X = add_egocentric_interaction_features(X, mouse_pair, avail_A, avail_B, fps)
            X = add_interaction_features(X, mouse_pair, avail_A, avail_B, fps)

    # nose-nose distance + close percentage
    if 'nose' in avail_A and 'nose' in avail_B:
        nn = np.sqrt(
            (mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
            (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2
        )
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nn_lg{lag}'] = nn.shift(l)
            X[f'nn_ch{lag}'] = nn - nn.shift(l)
            is_close = (nn < 10.0).astype(float)
            X[f'cl_ps{lag}'] = is_close.rolling(l, min_periods=1).mean()
    

    return X.astype(np.float32, copy=False)


# Preparation for training

In [17]:
def clean_and_fill_submission(submission, meta_df, is_train=True):
    if is_train:
        tracking_dir = CFG.train_tracking_path
    else: 
        tracking_dir = CFG.test_tracking_path
    
    # remove where start >= stop
    prev_len = len(submission)
    submission = submission[submission['start_frame'] < submission['stop_frame']].copy()
    if len(submission) != prev_len:
        print("Dropped rows with start_frame > stop_frame")
    
    # remove overlap
    prev_len = len(submission)
    cleaned_groups = []

    for (_, grp) in submission.groupby(['video_id', 'agent_id', 'target_id']):
        grp = grp.sort_values('start_frame')
        keep_mask = np.ones(len(grp), dtype=bool)

        last_stop = -1
        for i, (_, row) in enumerate(grp.iterrows()):
            if row['start_frame'] < last_stop:
                keep_mask[i] = False
            else:
                last_stop = row['stop_frame']
        
        cleaned_groups.append(grp[keep_mask])

    submission = pd.concat(cleaned_groups, ignore_index=True)
    if len(submission) != prev_len:
        print("Dropped rows with overlapped intervals")   

    # dummy prediction for video have no prediction
    dummy_rows = []

    for _, row in meta_df.iterrows():
        lab_id = row["lab_id"]

        # remove MABe22 vids
        if isinstance(lab_id, str) and lab_id.startswith("MABe22"):
            continue
        
        # remove behaviors_labeled if not string
        if not isinstance(row.get("behaviors_labeled", None), str):
            continue

        video_id = row["video_id"]

        # if have prediction -> skip
        if (submission["video_id"] == video_id).any():
            continue

        print(f"Video {video_id} has no predictions. Filling dummy segments...")

        # read tracking
        path = f"{tracking_dir}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)

        # get list (agent, target, action) from meta
        raw_behaviors = json.loads(row["behaviors_labeled"])
        cleaned = {b.replace("'", "") for b in raw_behaviors}
        triplets = [b.split(",") for b in sorted(cleaned)]
        beh_df = pd.DataFrame(triplets, columns=["agent", "target", "action"])

        # get total frames of this video
        start_frame = vid["video_frame"].min()
        stop_frame = vid["video_frame"].max() + 1
        total_frames = stop_frame - start_frame

        # divide uniformly 
        for (agent, target), actions in beh_df.groupby(["agent", "target"]):
            n_actions = len(actions)
            if n_actions == 0:
                continue

            batch_len = int(np.ceil(total_frames / n_actions))

            for i, (_, act_row) in enumerate(actions.iterrows()):
                batch_start = start_frame + i * batch_len
                batch_stop = min(batch_start + batch_len, stop_frame)

                dummy_rows.append((
                    video_id,
                    act_row["agent"],
                    act_row["target"],
                    act_row["action"],
                    batch_start,
                    batch_stop,
                ))

    if dummy_rows:
        dummy_df = pd.DataFrame(
            dummy_rows,
            columns=["video_id", "agent_id", "target_id", "action", "start_frame", "stop_frame"],
        )
        submission = pd.concat([submission, dummy_df], ignore_index=True)
        print(f"Filled {len(dummy_rows)} dummy segments for empty videos")

    return submission


In [18]:
def tune_thresholds_per_class(y_true, y_pred_prob, class_names):
    best_thrs = {}
    best_f1s = {}
    
    # tạo mảng ngưỡng (1 dòng, 99 cột)
    thresholds = np.arange(0.01, 1.00, 0.01)
    
    # loop qua từng class (không tránh được, nhưng bên trong sẽ rất nhanh)
    for i, name in enumerate(class_names):
        y_t = y_true[:, i]      # Shape: (N,)
        y_p = y_pred_prob[:, i] # Shape: (N,)
        
        # Nếu không có mẫu dương nào, skip ngay
        if y_t.sum() == 0: 
            best_thrs[name] = 0.5
            best_f1s[name] = 0.0
            continue
        
        # --- BƯỚC CẢI TIẾN: VECTOR HÓA ---
        
        # 1. So sánh y_p với TẤT CẢ ngưỡng cùng lúc
        # y_p[:, None] shape là (N, 1)
        # thresholds[None, :] shape là (1, 99)
        # Kết quả pred_matrix shape là (N, 99) - Ma trận True/False
        pred_matrix = (y_p[:, None] >= thresholds[None, :])
        
        # 2. Tính True Positive (TP), False Positive (FP), False Negative (FN)
        # Nhân với y_t[:, None] (Shape N, 1) để broadcast
        
        # TP: Dự đoán True VÀ Thực tế True
        tp = (pred_matrix & (y_t[:, None] == 1)).sum(axis=0)
        
        # FP: Dự đoán True NHƯNG Thực tế False
        fp = (pred_matrix & (y_t[:, None] == 0)).sum(axis=0)
        
        # FN: Dự đoán False NHƯNG Thực tế True
        # (~pred_matrix) đảo ngược True/False
        fn = ((~pred_matrix) & (y_t[:, None] == 1)).sum(axis=0)
        
        # 3. Tính F1 Score cho cả 99 ngưỡng cùng lúc theo công thức
        # F1 = 2TP / (2TP + FP + FN)
        epsilon = 1e-7 # Tránh chia cho 0
        f1_scores = 2 * tp / (2 * tp + fp + fn + epsilon)
        
        # 4. Tìm vị trí có F1 cao nhất
        best_idx = np.argmax(f1_scores)
        
        best_f1 = f1_scores[best_idx]
        best_thr = thresholds[best_idx]
        
        best_thrs[name] = float(best_thr)
        best_f1s[name] = float(best_f1)
        
    return best_thrs, best_f1s

In [19]:
def predict_multiclass(pred, meta, thresholds,action_names):
    # 1) Đảm bảo pred là DataFrame
    if isinstance(pred, np.ndarray):
        pred = pd.DataFrame(pred)

    pred_smoothed = pred.rolling(window=5, min_periods=1, center=True).mean()
    ama = np.argmax(pred_smoothed.values, axis=1)
    max_proba = pred_smoothed.max(axis=1).values

    threshold_array = np.asarray(thresholds)  
    action_thresholds = threshold_array[ama]


    ama = np.where(max_proba >= action_thresholds, ama, -1)
    ama = pd.Series(ama, index=meta.video_frame)

    # 7) Tìm điểm đổi action
    changes_mask = (ama != ama.shift(1)).values
    ama_changes = ama[changes_mask]
    meta_changes = meta[changes_mask]
    mask = ama_changes.values >= 0
    mask[-1] = False

    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'][mask].values,
        'agent_id': meta_changes['agent_id'][mask].values,
        'target_id': meta_changes['target_id'][mask].values,
        'action': [action_names[i] for i in ama_changes[mask].values],
        'start_frame': ama_changes.index[mask],
        'stop_frame': ama_changes.index[1:][mask[:-1]]
    })

    # 8) Xử lý stop_frame bị sai do nhảy video
    stop_video_id = meta_changes['video_id'][1:][mask[:-1]].values
    stop_agent_id = meta_changes['agent_id'][1:][mask[:-1]].values
    stop_target_id = meta_changes['target_id'][1:][mask[:-1]].values

    for i in range(len(submission_part)):
        video_id = submission_part.video_id.iloc[i]
        agent_id = submission_part.agent_id.iloc[i]
        target_id = submission_part.target_id.iloc[i]

        if i < len(stop_video_id):
            if (stop_video_id[i] != video_id or 
                stop_agent_id[i] != agent_id or 
                stop_target_id[i] != target_id):
                
                new_stop_frame = meta.query("video_id == @video_id").video_frame.max() + 1
                submission_part.at[i, 'stop_frame'] = new_stop_frame
        else:
            new_stop_frame = meta.query("video_id == @video_id").video_frame.max() + 1
            submission_part.at[i, 'stop_frame'] = new_stop_frame

    # 9) Loại event quá ngắn
    duration = submission_part.stop_frame - submission_part.start_frame
    submission_part = submission_part[duration >= 3].reset_index(drop=True)

    return submission_part


In [20]:
def tune_threshold(oof_action, y_action, step=0.01):
    best_thr = 0.5
    best_f1 = -1.0

    for thr in np.arange(0.0, 1.0 + 1e-9, step):
        y_pred = (oof_action >= thr).astype(int)
        f1 = f1_score(y_action, y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_thr = thr

    return float(best_thr)

In [21]:
import gc
import warnings
import joblib
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import f1_score


def cross_validate_classifier(X, label, meta, body_parts_str, section):
    oof = pd.DataFrame(index=meta.index)
    
    f1_list = []
    submission_list = []
    thresholds = {}

    # iter by action (binary)
    for action in label.columns:
        action_mask = ~ label[action].isna().values
        y_action = label[action][action_mask].values.astype(int)
        X_action = X[action_mask]
        groups_action = meta.video_id[action_mask]

        # not enough video for K-fold
        if len(np.unique(groups_action)) < CFG.n_splits:
            print(f"\tSkip (not enough groups). Section: {section} Action: {action}")
            continue

        # if all label is 0 -> not meaningful
        if (y_action == 0).all():
            oof_action = np.zeros(len(y_action), dtype=float)
            print(f"\tF1: 0.0000 (0.00) Section: {section} Action: {action}")
        else:
            try:
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore", category=RuntimeWarning)

                    trainer = Trainer(
                        estimator=clone(CFG.model),
                        cv=CFG.cv,
                        cv_args={"groups": groups_action},
                        metric=f1_score,
                        task="binary",
                        verbose=False,
                        save=True,
                        save_path=f"{CFG.model_name}/{section}/{action}",
                        use_early_stopping=True,   
                    )


                    trainer.fit(
                        X_action,
                        y_action,
                        fit_args={
                            "early_stopping_rounds": 200,    
                            "eval_metric": "aucpr",      
                            "verbose": False             
                        }
                    )
                    
                    oof_action = trainer.oof_preds  # shape [N_action_samples]

                    # tune threshold for this action
                    threshold = tune_threshold(oof_action, y_action)
                    thresholds[action] = threshold

                    f1 = f1_score(y_action, (oof_action >= threshold).astype(int), zero_division=0)
                    f1_list.append((body_parts_str, action, f1))

                    # save OOF and threshold
                    joblib.dump(oof_action, f"{CFG.model_name}/{section}/{action}/oof_pred_probs.pkl")
                    joblib.dump(threshold, f"{CFG.model_name}/{section}/{action}/threshold.pkl")

                    print(f"\tF1: {f1:.4f} (thr={threshold:.2f}) "
                          f"Section: {section} Action: {action}")

                del trainer
                gc.collect()

            except Exception as e:
                oof_action = np.zeros(len(y_action), dtype=float)
                thresholds[action] = 0.5  # default backup
                print(f"\tERROR: {e} -> F1: 0.0000 (0.50) Section: {section} Action: {action}")

        oof_column = np.zeros(len(label), dtype=float)
        oof_column[action_mask] = oof_action
        oof[action] = oof_column

        del oof_action, action_mask, X_action, y_action, groups_action
        gc.collect()

    submission_part = predict_multiclass(oof, meta, thresholds)
    print(submission_part)
    submission_list.append(submission_part)

    return submission_list, f1_list, thresholds


In [22]:
def submit(test_subset, fps_lookup, body_parts, mode, section, thresholds):
    
    sample_gen = generate_mouse_data(test_subset, mode, is_train=False)
    submission_list = []
    for sample_mode, track_df, meta_df, actions in sample_gen:
        assert sample_mode == mode

        try:
            fps = _fps_from_meta(meta_df, fps_lookup, default_fps=30)
            if sample_mode == "single":
                X_te = transform_single(track_df, body_parts, fps)
            else:
                X_te = transform_pair(track_df, body_parts, fps)
            
            del track_df
            gc.collect()

            # get prediction
            pred = pd.DataFrame(index=meta_df.video_frame)

            # get model for each action with this section
            for action in actions:
                model_dir = f"{CFG.model_path}/{CFG.model_name}/{section}/{action}"
                model_files = glob.glob(f"{model_dir}/*_trainer_*.pkl")

                if len(model_files) != 1:
                    continue

                trainer = joblib.load(model_files[0])
                pred[action] = trainer.predict(X_te)

                del trainer
                gc.collect()

            del X_te
            gc.collect()

            # if have columns, convert frame-wise prob to intervals
            if pred.shape[1] > 0:
                submission = predict_multiclass(pred, meta_df, thresholds)
                submission_list.append(submission)

        except KeyError:
            del track_df
            gc.collect()

        
    return submission_list


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import gc
import os
import joblib
import json
import warnings

# Tắt warning
warnings.filterwarnings('ignore')

MODEL_DIR = getattr(CFG, "model_dir", CFG.model_name)
if not os.path.exists(MODEL_DIR): os.makedirs(MODEL_DIR)


# --- CLASS: EARLY STOPPING ---
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_f1, model, path):
        score = val_f1
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_f1, model, path)
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_f1, model, path)
            self.counter = 0

    def save_checkpoint(self, val_f1, model, path):
        torch.save(model.state_dict(), path)

# --- LOSS: FOCAL LOSS ---
class FocalBCEWithLogitsLoss(nn.Module):
    def __init__(self, pos_weight=None, gamma=2.0, reduction='mean'):
        super().__init__()
        self.pos_weight = pos_weight
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, logits, targets):
        bce_loss = F.binary_cross_entropy_with_logits(
            logits, targets, pos_weight=self.pos_weight, reduction='none'
        )
        probs = torch.sigmoid(logits)
        pt = torch.where(targets == 1, probs, 1 - probs)
        loss = (1 - pt) ** self.gamma * bce_loss
        
        if self.reduction == 'mean': return loss.mean()
        elif self.reduction == 'sum': return loss.sum()
        else: return loss

# ==========================================
# 1. PARAMETERS
# ==========================================
WINDOW_SIZE = 30  
BATCH_SIZE = 128
EPOCHS = 1       
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ==========================================
# 2. DATASET
# ==========================================
class MABeLazyDataset(Dataset):
    def __init__(self, feat_list_np, scaler, label_list=None, window_size=30):
        self.window_size = window_size
        self.feat_list = feat_list_np   
        self.label_list = label_list
        self.scaler = scaler
        
        # Tạo index map
        self.index_map = []
        for v_idx, arr in enumerate(self.feat_list):
            length = len(arr)
            if length >= window_size:
                self.index_map.extend([(v_idx, t) for t in range(length - window_size)])
    
    def __len__(self): 
        return len(self.index_map)
    
    def __getitem__(self, idx):
        v_idx, t = self.index_map[idx]
        
        # Slicing trên Numpy nhanh hơn Pandas .iloc rất nhiều
        window = self.feat_list[v_idx][t : t + self.window_size]
        
        # Scale "On-the-fly"
        if self.scaler is not None:
            window = self.scaler.transform(window)
        
        # Transpose: (Seq, Feat) -> (Feat, Seq)
        X_out = torch.tensor(window, dtype=torch.float32).transpose(0, 1)
        
        if self.label_list is not None:
            center_frame_idx = t + self.window_size // 2
            y_out = torch.tensor(self.label_list[v_idx][center_frame_idx], dtype=torch.float32)
            return X_out, y_out
        else: 
            return X_out

# ==========================================
# 3. MODEL: RESNET-1D
# ==========================================
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1, dropout=0.2):
        super().__init__()
        padding = (kernel_size - 1) * dilation // 2
        
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=padding, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        residual = self.shortcut(x)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.dropout(x)
        x = self.bn2(self.conv2(x))
        x += residual
        return self.relu(x)

class MouseResNet1D(nn.Module):
    def __init__(self, n_feat, n_class, base_filters=64):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv1d(n_feat, base_filters, kernel_size=7, padding=3),
            nn.BatchNorm1d(base_filters), nn.ReLU()
        )
        self.layer1 = ResidualBlock1D(base_filters, base_filters, dilation=1)
        self.layer2 = ResidualBlock1D(base_filters, base_filters*2, dilation=2)
        self.layer3 = ResidualBlock1D(base_filters*2, base_filters*4, dilation=4)
        
        self.global_pool = nn.AdaptiveAvgPool1d(1) 
        self.fc = nn.Linear(base_filters*4, n_class)
        
    def forward(self, x):
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.global_pool(x).squeeze(-1)
        return self.fc(x)

# ==========================================
# 4. TRAINING FUNCTION (GPU OPTIMIZED)
# ==========================================
def train_evaluate_cnn(feat_list, label_list, meta_list, section, mode="single", n_splits=3):
    if not os.path.exists(MODEL_DIR): 
        os.makedirs(MODEL_DIR)

    print(">>> Converting DataFrames to Numpy ...")

    # Union features
    ref_cols = sorted({c for df in feat_list for c in df.columns})
    save_path_cols = f"{MODEL_DIR}/cols_sec{section}_{mode}.json"
    json.dump(ref_cols, open(save_path_cols, "w"))


    # Convert toàn bộ feats
    feat_list_np = []
    for df in feat_list:
        arr = df.reindex(columns=ref_cols, fill_value=0).values.astype(np.float32)
        arr = np.nan_to_num(arr, nan=0.0, posinf=0.0, neginf=0.0)
        feat_list_np.append(arr)

    del feat_list
    gc.collect()

    # Labels
    if label_list:
        all_actions = set()
        for df in label_list:
            all_actions.update(df.columns)
        ref_label_cols = sorted(list(all_actions))

        label_list_np = []
        for df in label_list:
            label_list_np.append(
                df.reindex(columns=ref_label_cols, fill_value=0).values.astype(np.float32)
            )
    else:
        label_list_np = None
        ref_label_cols = ["dummy"]

    # Fold split
    video_indices = np.arange(len(feat_list_np))
    video_labels = []

    if label_list_np:
        for arr in label_list_np:
            counts = arr.sum(axis=0)
            video_labels.append(np.argmax(counts) if counts.sum() > 0 else -1)
    else:
        video_labels = [0] * len(feat_list_np)

    video_groups = video_indices
    sgkf = StratifiedGroupKFold(n_splits=n_splits)

    fold_scores = []
    model = None
    scaler = None

    # ---------------- FOLDS ----------------
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(video_indices, video_labels, video_groups)):
        print(f"\n==============================")
        print(f">>> TRAINING FOLD {fold+1}/{n_splits}")
        print(f"==============================")

        train_feats = [feat_list_np[i] for i in train_idx]
        val_feats = [feat_list_np[i] for i in val_idx]

        train_lbls = [label_list_np[i] for i in train_idx] if label_list_np else None
        val_lbls = [label_list_np[i] for i in val_idx] if label_list_np else None

        # Scaling
        scaler = StandardScaler()
        for arr in train_feats:
            scaler.partial_fit(arr)

        ds_train = MABeLazyDataset(train_feats, scaler, train_lbls, window_size=WINDOW_SIZE)
        ds_val = MABeLazyDataset(val_feats, scaler, val_lbls, window_size=WINDOW_SIZE)

        loader_train = DataLoader(ds_train, batch_size=1024, shuffle=True, num_workers=4, pin_memory=True)
        loader_val = DataLoader(ds_val, batch_size=2048, shuffle=False, num_workers=4, pin_memory=True)

        # pos_weight
        pos_counts = np.zeros(len(ref_label_cols))
        total_samples = 0

        if train_lbls:
            for arr in train_lbls:
                if len(arr) >= WINDOW_SIZE:
                    valid_lbls = arr[WINDOW_SIZE//2 : -(WINDOW_SIZE//2)]
                    pos_counts += valid_lbls.sum(axis=0)
                    total_samples += len(valid_lbls)

        raw_weights = (total_samples - pos_counts) / (pos_counts + 1e-6)
        raw_weights = np.clip(raw_weights, 1.0, 100.0)

        pos_weight_val = torch.tensor(raw_weights, dtype=torch.float32).to(DEVICE)

        # Model
        n_feat_in = len(ref_cols)
        model = MouseResNet1D(n_feat=n_feat_in, n_class=len(ref_label_cols)).to(DEVICE)

        optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
        criterion = FocalBCEWithLogitsLoss(pos_weight=pos_weight_val, gamma=1.0)
        scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=False)

        save_path = f"{MODEL_DIR}/model_sec{section}_{mode}_fold{fold}.pth"
        early_stopping = EarlyStopping(patience=CFG.patience, min_delta=0.001)

        # ---------------- TRAIN ----------------
        for epoch in range(EPOCHS):
            model.train()
            train_loss = 0

            for x_batch, y_batch in loader_train:
                x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)

                optimizer.zero_grad()
                pred = model(x_batch)
                loss = criterion(pred, y_batch)
                if torch.isnan(loss):
                    continue
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                train_loss += loss.item()

            # Validation
            model.eval()
            val_probs, val_true = [], []
            with torch.no_grad():
                for x_b, y_b in loader_val:
                    val_probs.append(torch.sigmoid(model(x_b.to(DEVICE))).cpu())
                    val_true.append(y_b)

            vp = torch.cat(val_probs).numpy()
            vt = torch.cat(val_true).numpy()

            mean_f1 = f1_score(vt, (vp > 0.5).astype(int), average='macro', zero_division=0)
            avg_train_loss = train_loss / len(loader_train)

            print(f"Ep {epoch+1}: Loss {avg_train_loss:.4f} | Val F1: {mean_f1:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f}")

            scheduler.step(mean_f1)
            early_stopping(mean_f1, model, save_path)

            if early_stopping.early_stop:
                print("Early stopping triggered!")
                break

        # ---------------- FINAL EVAL ----------------
        model.load_state_dict(torch.load(save_path))
        model.eval()

        val_probs, val_true = [], []
        with torch.no_grad():
            for x_b, y_b in loader_val:
                val_probs.append(torch.sigmoid(model(x_b.to(DEVICE))).cpu())
                val_true.append(y_b)

        vp, vt = torch.cat(val_probs).numpy(), torch.cat(val_true).numpy()

        print(f"\n--- Threshold Tuning Fold {fold+1} ---")
        best_thrs, best_f1s = tune_thresholds_per_class(vt, vp, ref_label_cols)
        fold_scores.append(np.mean(list(best_f1s.values())))

        for k, v in best_f1s.items():
            print(f"{k}: F1={v:.4f} | thr={best_thrs[k]:.2f}")

        with open(f"{MODEL_DIR}/thresholds_sec{section}_{mode}_fold{fold}.json", "w") as f:
            json.dump(best_thrs, f)

        joblib.dump(scaler, f"{MODEL_DIR}/scaler_sec{section}_{mode}_fold{fold}.pkl")

        # Cleanup
        del optimizer, loader_train, loader_val, ds_train, ds_val
        del train_feats, val_feats, train_lbls, val_lbls

        if fold < n_splits - 1:
            del model
            del scaler

        torch.cuda.empty_cache()
        gc.collect()

    print(f"\n>>> MEAN F1 OVER {n_splits} FOLDS: {np.mean(fold_scores):.4f} <<<")
    return model, scaler


In [24]:

import torch
import numpy as np
import pandas as pd
import joblib
import json
import os
import gc
from tqdm import tqdm

# ==========================================
# 1. MODEL + SCALER
# ==========================================
def load_resnet_resources(section, mode="single", n_folds=3, device='cuda'):
    print(f"Loading ResNet Ensemble ({n_folds} folds) | Section {section} | Mode = {mode}")
    pack = {
        'scalers': [],
        'models': [],
        'thresholds': {},
        'n_feat': 0,
        'action_names': [],
        'feature_cols': [],
        'current_feature_names': []
    }

    # Model directory
    MODEL_DIR = getattr(CFG, 'model_dir', CFG.model_name)
    alt_dir = f"{getattr(CFG, 'model_path', '.').rstrip('/')}/{getattr(CFG, 'model_name', '1d-cnn')}"
    base_dir = MODEL_DIR if os.path.exists(MODEL_DIR) else alt_dir

    # ====================================================
    # A. Load FEATURE COLS
    # ====================================================
    feat_col_path = f"{base_dir}/cols_sec{section}_{mode}.json"
    if not os.path.exists(feat_col_path):
        raise FileNotFoundError(
            f"Missing feature_cols file: {feat_col_path}\n"
            "Did you save feature columns using mode-specific path?"
        )

    with open(feat_col_path, "r") as f:
        pack['feature_cols'] = json.load(f)

    # ====================================================
    # B. Load MODELS + SCALERS
    # ====================================================
    for fold in range(n_folds):
        scaler_path = f"{base_dir}/scaler_sec{section}_{mode}_fold{fold}.pkl"
        if not os.path.exists(scaler_path):
            raise FileNotFoundError(f"Missing scaler: {scaler_path}")

        pack['scalers'].append(joblib.load(scaler_path))

        model_path = f"{base_dir}/model_sec{section}_{mode}_fold{fold}.pth"
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Missing model: {model_path}")

        # Load state dict
        state = torch.load(model_path, map_location=device)

        # Detect n_feat, n_class from model weights
        pack['n_feat'] = state['stem.0.weight'].shape[1]
        last_key = [k for k in state.keys() if 'weight' in k][-1]
        n_class = state[last_key].shape[0]

        model = MouseResNet1D(
            n_feat=pack['n_feat'],
            n_class=n_class,
            base_filters=64
        ).to(device)

        model.load_state_dict(state)
        model.eval()
        pack['models'].append(model)

    # ====================================================
    # C. Load AVG THRESHOLDS
    # ====================================================
    avg_thrs = {}
    counts = {}
    json_found = False

    for fold in range(n_folds):
        thr_path = f"{base_dir}/thresholds_sec{section}_{mode}_fold{fold}.json"
        if os.path.exists(thr_path):
            json_found = True
            with open(thr_path, "r") as f:
                thrs = json.load(f)

            for k, v in thrs.items():
                avg_thrs[k] = avg_thrs.get(k, 0.0) + v
                counts[k] = counts.get(k, 0) + 1

    if not json_found:
        print("⚠ Warning: No thresholds found, using default 0.5")
        dummy = [f"action_{i}" for i in range(n_class)]
        avg_thrs = {k: 0.5 for k in dummy}
        pack['action_names'] = dummy
    else:
        for k in avg_thrs:
            avg_thrs[k] /= counts[k]
        pack['action_names'] = list(avg_thrs.keys())

    pack['thresholds'] = avg_thrs

    # ====================================================
    # D. Set Current Feature Names
    # ====================================================
    pack["current_feature_names"] = pack["feature_cols"]

    print(f"-> Loaded feature columns: {len(pack['feature_cols'])}")
    print(f"-> n_feat = {pack['n_feat']} | n_class = {len(pack['action_names'])}")
    print(f"-> Loaded {len(pack['models'])} models for mode={mode}")

    return pack

# ==========================================
# 2. FIX VIDEO ID & PADDING
# ==========================================
def submit(test_subset, fps_lookup, body_parts, mode, section, thresholds=None, is_train=False):
    submission_list = []
    device = "cuda" if torch.cuda.is_available() else "cpu"
    window_size = WINDOW_SIZE 

    # 1. Load Resources
    try:
        pack = load_resnet_resources(section,mode, n_folds=CFG.n_splits, device=device)
        action_names = pack['action_names']
        final_thrs = pack['thresholds']
        expected_feat = pack['n_feat']
    except Exception as e:
        print(f"Error loading models for Sec {section}: {e}")
        return []

    # 2. Loop qua video
    sample_gen = generate_mouse_data(test_subset, mode, is_train=is_train)

    for _, track_df, meta_df, actions in tqdm(sample_gen, desc=f"Infer Sec {section}"):
        if is_train and not isinstance(actions, (list, tuple)):
            actions = list(actions.columns)

        video_id = meta_df["video_id"].iloc[0] if "video_id" in meta_df.columns else "Unknown"
        agent_id = meta_df["agent_id"].iloc[0] if "agent_id" in meta_df.columns else "unknown_agent"
        target_id = meta_df["target_id"].iloc[0] if "target_id" in meta_df.columns else "unknown_target"

        try:
            # --- Feature Engineering ---
            fps = _fps_from_meta(meta_df, fps_lookup, default_fps=30)
            if mode == "single":
                X = transform_single(track_df, body_parts, fps)
            else:
                X = transform_pair(track_df, body_parts, fps)

            del track_df
            gc.collect()


            ref_cols = pack["feature_cols"]
            X_df = pd.DataFrame(X, columns=pack["current_feature_names"])
            X_df = X_df.reindex(columns=ref_cols, fill_value=0)
            X = X_df.values.astype(np.float32)



            n_frames = len(X)
            full_preds = np.zeros((n_frames, len(action_names)), dtype=np.float32)
        

            if n_frames < window_size:
                ols = ['video_id', 'agent_id', 'target_id', 'frame'] + action_names
                df_res = pd.DataFrame(columns=cols)
                df_res['frame'] = range(n_frames)
                df_res['video_id'] = video_id
                df_res['agent_id'] = agent_id
                df_res['target_id'] = target_id
                df_res[action_names] = 0
                submission_list.append(df_res)
                continue

            # --- ENSEMBLE PREDICTION ---
            accum_probs = 0
            n_windows = max(1, n_frames - window_size + 1)
            n_folds = len(pack["models"])
            for i in range(n_folds):
                scaler = pack['scalers'][i]
                model = pack['models'][i]

                X_scaled = scaler.transform(np.nan_to_num(X))
                X_scaled = np.nan_to_num(X_scaled)

                windows = np.array([X_scaled[t : t+window_size] for t in range(n_windows)])
                inp = torch.tensor(windows, dtype=torch.float32).transpose(1, 2).to(device)

                
                with torch.no_grad():
                    probs = torch.sigmoid(model(inp)).cpu().numpy()
                    accum_probs += probs

            avg_probs = accum_probs / float(n_folds)

            
            thr_array = np.array([final_thrs.get(action, 0.5) for action in action_names])

            offset = window_size // 2
            full_preds[offset : offset + len(avg_probs)] = avg_probs
            full_preds[:offset] = avg_probs[0]
            full_preds[offset + len(avg_probs):] = avg_probs[-1]


            if full_preds.shape[1] > 0:
                submission = predict_multiclass(full_preds, meta_df, thr_array,action_names)
                submission_list.append(submission)


            del X, windows, inp, avg_probs

        except Exception as e:
            print(f"!!! Error processing video {video_id}: {e}")
            gc.collect()
            continue
    return submission_list


In [25]:

import os 

def process_mode(mode, subset, body_parts, fps_lookup, section, thresholds, f1_list, submission_list):
    # validate or test
    if CFG.mode == "validate":
        data_list, label_list, meta_list = [], [], []

        for switch, data, meta, label in generate_mouse_data(subset):
            if switch != mode:
                continue
            data_list.append(data)
            meta_list.append(meta)
            label_list.append(label)
            del data, meta, label
        gc.collect()

        if len(data_list) == 0:
            return  

        # features for each sample
        feats_parts = []
        for data_i, meta_i in zip(data_list, meta_list):
            fps_i = _fps_from_meta(meta_i, fps_lookup, default_fps=30.0)
            if mode == "single":
                X_i = transform_single(data_i, body_parts, fps_i)
            else:
                X_i = transform_pair(data_i, body_parts, fps_i)

            feats_parts.append(X_i.astype(np.float32))
            del X_i, fps_i
        gc.collect()

        # --- CNN ---
        print(f"Start training CNN for section {section}")

        # training CNN
        model, scaler = train_evaluate_cnn(feats_parts, label_list, meta_list, section,mode)

        # Load thresholds
        try:
            pack = load_resnet_resources(section, n_folds=CFG.n_splits, device=("cuda" if torch.cuda.is_available() else "cpu"))
            thresholds[mode].setdefault(str(section), {})
            thresholds[mode][str(section)] = pack.get("thresholds", {})
        except Exception as e:
            print(f"Warning: could not load thresholds for section {section}: {e}")

        # Sau khi train, chạy inference trên tập train (validate) sinh submission_list
        temp_sub_list = submit(
            test_subset=subset,
            body_parts=body_parts,
            fps_lookup=fps_lookup,
            mode=mode,
            section=section,
            thresholds=thresholds,
            is_train=True
        )
        for df in temp_sub_list:
            submission_list.append(df)

        # Cleanup
        del feats_parts, label_list, meta_list
        gc.collect()

    else:
        print(f"Predicting Section {section} using ResNet Ensemble...")

        temp_sub_list = submit(
            test_subset=subset,
            body_parts=body_parts,
            fps_lookup=fps_lookup,
            mode=mode,
            section=section,
            thresholds=None,
            is_train=False
        )

        for df in temp_sub_list:
            submission_list.append(df)

        del temp_sub_list; gc.collect()


In [26]:
thresholds = {
        "single": {},
        "pair": {}
    }

In [27]:
f1_list = []
submission_list = []
import traceback

# for section in range(1, len(body_parts_tracked_list)):
for section in range(1, len(body_parts_tracked_list)):
    body_parts_tracked_str = body_parts_tracked_list[section]

    try:
        body_parts = json.loads(body_parts_tracked_str)
        print(f"{section}/{len(body_parts_tracked_list)-1} Processing videos with: {body_parts}\n")

        if len(body_parts) > 5:
            body_parts = [b for b in body_parts if b not in DROP_BODY_PARTS]

        if CFG.mode == "validate":
            subset =  train[train.body_parts_tracked == body_parts_tracked_str]
        else:
            subset = test[test.body_parts_tracked == body_parts_tracked_str]

        # lookup FPS
        fps_lookup = (
            subset[["video_id", "frames_per_second"]]
            .drop_duplicates("video_id")
            .set_index("video_id")["frames_per_second"]
            .to_dict()
        )

        # single
        process_mode(
            mode="single",
            subset=subset,
            body_parts=body_parts,
            fps_lookup=fps_lookup,
            section=section,
            thresholds=thresholds,
            f1_list=f1_list,
            submission_list=submission_list,
        )

        # pair
        process_mode(
            mode="pair",
            subset=subset,
            body_parts=body_parts,
            fps_lookup=fps_lookup,
            section=section,
            thresholds=thresholds,
            f1_list=f1_list,
            submission_list=submission_list,
        )

        print(f"Length of submission_list: {len(submission_list)}\n")

    except Exception as e:
        print(f"\tError: {e}")



1/9 Processing videos with: ['body_center', 'ear_left', 'ear_right', 'headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 'lateral_left', 'lateral_right', 'neck', 'nose', 'tail_base', 'tail_midpoint', 'tail_tip']

Predicting Section 1 using ResNet Ensemble...
Loading ResNet Ensemble (3 folds) | Section 1 | Mode = single
-> Loaded feature columns: 142
-> n_feat = 142 | n_class = 1
-> Loaded 3 models for mode=single


Infer Sec 1: 4it [02:31, 37.82s/it]


Predicting Section 1 using ResNet Ensemble...
Loading ResNet Ensemble (3 folds) | Section 1 | Mode = pair
-> Loaded feature columns: 164
-> n_feat = 164 | n_class = 6
-> Loaded 3 models for mode=pair


Infer Sec 1: 12it [07:42, 38.55s/it]


Length of submission_list: 16

2/9 Processing videos with: ['body_center', 'ear_left', 'ear_right', 'hip_left', 'hip_right', 'lateral_left', 'lateral_right', 'nose', 'spine_1', 'spine_2', 'tail_base', 'tail_middle_1', 'tail_middle_2', 'tail_tip']

Predicting Section 2 using ResNet Ensemble...
Loading ResNet Ensemble (3 folds) | Section 2 | Mode = single
-> Loaded feature columns: 151
-> n_feat = 151 | n_class = 3
-> Loaded 3 models for mode=single


...

Infer Sec 9: 0it [00:00, ?it/s]

Predicting Section 9 using ResNet Ensemble...
Loading ResNet Ensemble (3 folds) | Section 9 | Mode = pair


-> Loaded feature columns: 43
-> n_feat = 43 | n_class = 5
-> Loaded 3 models for mode=pair


Infer Sec 9: 0it [00:00, ?it/s]

Length of submission_list: 16



In [28]:
if CFG.mode == 'validate':  
    submission = pd.concat(submission_list)
    cleaned_submission = clean_and_fill_submission(submission, train)
    print(f"Competition metric: {score(solution, cleaned_submission, ''):.4f}")

    f1_df = pd.DataFrame(f1_list, columns=['body_parts_tracked_str', 'action', 'binary F1 score'])
    print(f"Mean F1:            {f1_df['binary F1 score'].mean():.4f}")
  
    joblib.dump(thresholds, f"{CFG.model_name}/thresholds.pkl")
    joblib.dump(f1_df, f"{CFG.model_name}/scores.pkl")

elif CFG.mode == 'submit':
    if len(submission_list) > 0:
        submission = pd.concat(submission_list)
    else:
        submission = pd.DataFrame(
            dict(
                video_id=438887472,
                agent_id='mouse1',
                target_id='self',
                action='rear',
                start_frame=278,
                stop_frame=500
            ), index=[44])

    cleaned_submission = clean_and_fill_submission(submission, test, is_train=False)
    cleaned_submission.index.name = 'row_id'
    cleaned_submission.to_csv('submission.csv')
    submission.head()